In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, LlamaForCausalLM

base_model_path = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    #quantization_config=bnb_config,
    #device_map="auto",
    use_cache=False
)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [2]:
tokenizer = AutoTokenizer.from_pretrained(base_model_path)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [5]:
text = "Sample input."

input_ids = tokenizer(text, return_tensors="pt").input_ids
%time output = model(input_ids)
print(output.logits)

CPU times: user 1.9 s, sys: 0 ns, total: 1.9 s
Wall time: 124 ms
tensor([[[ -4.6822,   0.9866,   4.5126,  ...,  -5.2010,  -2.1646,  -4.2286],
         [-10.1421, -10.1378,   5.6231,  ...,  -6.1040, -11.0720,  -4.8641],
         [ -9.6640,  -9.2623,   4.9745,  ...,  -7.9037, -11.2208,  -3.2938],
         [ -8.7681,  -8.3673,   8.2792,  ...,  -5.3513,  -8.6309,  -3.2394]]],
       grad_fn=<UnsafeViewBackward0>)


In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'
from transformers.models.zamba2 import Zamba2Config, Zamba2Model
config = Zamba2Config(# hidden_size=256,
                    use_cache=False,
                    use_mamba_kernels=True)

model = Zamba2Model(config)
model = model.cuda()


In [2]:
import torch
inp = torch.tensor([[1, 2]]).cuda()
model(inp)

torch.Size([2, 1, 5120])
<function MLP.__init__.<locals>.glu at 0x7fbbf42c8700>
tensor([[[-0.2819,  0.8015,  0.4129,  ...,  0.4797,  1.4796,  0.8483],
         [-0.2460, -0.1759,  1.7739,  ..., -1.7110,  1.0436,  0.6792]]],
       device='cuda:0', grad_fn=<AddBackward0>)
torch.Size([2, 1, 5120])
<function MLP.__init__.<locals>.glu at 0x7fba696c60e0>
tensor([[[-1.0465, -2.1433,  2.8452,  ..., -0.7424, -0.3116, -0.0337],
         [ 0.4994, -2.6086,  0.6702,  ..., -2.1001,  0.5389, -1.2339]]],
       device='cuda:0', grad_fn=<AddBackward0>)
torch.Size([2, 1, 5120])
<function MLP.__init__.<locals>.glu at 0x7fbbf42c8700>
tensor([[[ 1.2252,  0.2309,  1.2017,  ...,  0.8811,  0.8888, -0.8302],
         [ 0.0926,  0.8126, -0.1900,  ...,  0.5705,  1.3784, -1.2281]]],
       device='cuda:0', grad_fn=<AddBackward0>)
torch.Size([2, 1, 5120])
<function MLP.__init__.<locals>.glu at 0x7fba696c60e0>
tensor([[[-1.3620, -1.1314,  1.5622,  ..., -2.2054,  0.3344, -1.4363],
         [-0.3542, -1.3665,  2.47

BaseModelOutputWithPast(last_hidden_state=tensor([[[-0.9624, -2.4215, -0.4542,  ...,  0.4463,  2.5638,  0.6742],
         [ 0.8970,  0.5811, -1.5586,  ..., -1.2077, -0.4730,  0.8435]]],
       device='cuda:0', grad_fn=<MulBackward0>), past_key_values=None, hidden_states=None, attentions=None)

# Checkpoint to .bin

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'

import torch
# from transformers import ZambaForCausalLM, ZambaConfig, AutoTokenizer
# from transformers.models.jamba.modeling_jamba import HybridMambaAttentionDynamicCache
torch.cuda.device_count()

1

In [ ]:
from megatron import checkpointing
import sys, torch, os
from megatron.initialize import initialize_megatron

# CONFIG_PATH = "/workspace/mamba_memory_block_7B_annealing_nochat_noTP.sh"
# CHECKPOINT_PATH = "/checkpoints/mamba_memory_block_7B_annealing_nochat_reformatted_openOrca_noTP"
# ITERATION = 25156

CONFIG_PATH = "/workspace/Mamba-MoE/examples/mamba_memory_block/zamba_3B_annealing_final.sh"
CHECKPOINT_PATH = "/checkpoints/zamba_3B_attempt_2"
ITERATION = 462070


def extract_keyword_args(filestr, keyword):
    gpt_split = filestr.split(keyword)
    if len(gpt_split) <=1:
        raise ValueError("Config provided does not have a GPT_ARGS variable provided")
    arg_splits = gpt_split[1].split("\"")
    gpt_args = arg_splits[1]
    gpt_args = gpt_args.replace("\n","").replace("\\","").replace("\t","")
    gpt_args = ' '.join(gpt_args.split())
    return gpt_args.strip().split(" ")


def extract_data_paths(filestr, CHECKPOINT_PATH):
    tokenizer_type = filestr.split("TOKENIZER_TYPE=")[1].split("\n")[0]
    tokenizer_model = filestr.split("TOKENIZER_MODEL=")[1].split("\n")[0]

    data_path = filestr.split("DATA_PATH=")[1].split("\n")[0]
    return ["--data-path", data_path, "--tokenizer-type", tokenizer_type, "--hf_autotokenizer_model", tokenizer_model, "--load", CHECKPOINT_PATH]

def parse_config_file_update_argv(CONFIG_PATH, CHECKPOINT_PATH):
    with open(CONFIG_PATH,"r") as f:
        filestr = f.read()
    
    sys.argv = [""]
    sys.argv += extract_keyword_args(filestr, "GPT_ARGS")
    sys.argv += extract_data_paths(filestr, CHECKPOINT_PATH)

def init_megatron():
    initialize_megatron(
        args_defaults={
            'tokenizer_type': 'HFAutoTokenizer',
            'hf_autotokenizer_model': 'EleutherAI/gpt-neox-20b',
            'no_load_rng': True,
            'no_load_optim': True
        }
    )



os.environ["CUDA_DEVICE_MAX_CONNECTIONS"] = "1"
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "6000"
parse_config_file_update_argv(CONFIG_PATH, CHECKPOINT_PATH)
init_megatron()

# name = checkpointing.get_checkpoint_name(
#     CHECKPOINT_PATH, 
#     iteration=ITERATION,
#     #tensor_rank=i,
# )

In [2]:
name = '/checkpoints/zamba_3B_attempt_2/final_anneal/mp_rank_00/model_optim_rng.pt'
print(f"Loading {name}")
#state_dicts.append(torch.load(name, map_location='cpu'))
state_dict = torch.load(name, map_location='cuda:0')

Loading /checkpoints/zamba_3B_attempt_2/final_anneal/mp_rank_00/model_optim_rng.pt


In [3]:
# Save state dictionary

import pickle
dict_1 = state_dict['model']
with open('/workspace/transformers_zamba2_dev/src/transformers/models/zamba2/state_dict.pkl', 'wb') as pickle_file:
    pickle.dump(dict_1, pickle_file)

In [1]:
# Load state dictionary
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'
import pickle
with open('/workspace/transformers_zamba2_dev/src/transformers/models/zamba2/state_dict.pkl', 'rb') as pickle_file:
    dict_1 = pickle.load(pickle_file)

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'
from transformers.models.zamba2 import Zamba2Config, Zamba2ForCausalLM
config = Zamba2Config()
model = Zamba2ForCausalLM(config).cuda()
model = model.cuda()

In [3]:
for key, value in dict_1.items():
    if 'decoder.layers.' not in key and 'decoder.blocks' not in key:
        print(key , value.shape)
print()
for n, p in model.named_parameters():
    if 'mamba_layers.' not in n and 'blocks.' not in n:
        print(n, p.shape)

count = 0
for n, p in model.named_parameters():
    if 'mamba_layers.' not in n and 'blocks.' not in n:
        if 'embed_tokens' in n:
            assert p.data.shape == dict_1['embedding.word_embeddings.weight'].shape
            p.data = dict_1['embedding.word_embeddings.weight'].clone().detach()
        if 'final_layernorm' in n:
            assert p.data.shape == dict_1['decoder.final_layernorm.weight'].shape
            p.data = dict_1['decoder.final_layernorm.weight'].clone().detach()
        if 'linear_layers' in n:
            num = int(n[20])
            num2num = [6, 12, 18, 24, 30, 36, 42, 47, 51]
            name = 'decoder.block_map.'+str(num2num[num])+'.weight'
            assert p.data.shape == dict_1[name].shape
            p.data = dict_1[name].clone().detach()

embedding.word_embeddings.weight torch.Size([32000, 2560])
embedding.position_embeddings.weight torch.Size([4096, 2560])
decoder.block_map.6.weight torch.Size([2560, 2560])
decoder.block_map.12.weight torch.Size([2560, 2560])
decoder.block_map.18.weight torch.Size([2560, 2560])
decoder.block_map.24.weight torch.Size([2560, 2560])
decoder.block_map.30.weight torch.Size([2560, 2560])
decoder.block_map.36.weight torch.Size([2560, 2560])
decoder.block_map.42.weight torch.Size([2560, 2560])
decoder.block_map.47.weight torch.Size([2560, 2560])
decoder.block_map.51.weight torch.Size([2560, 2560])
decoder.final_layernorm.weight torch.Size([2560])
decoder.buffer_params_1.weight torch.Size([1, 48])

model.embed_tokens.weight torch.Size([32000, 2560])
model.linear_layers.0.weight torch.Size([2560, 2560])
model.linear_layers.1.weight torch.Size([2560, 2560])
model.linear_layers.2.weight torch.Size([2560, 2560])
model.linear_layers.3.weight torch.Size([2560, 2560])
model.linear_layers.4.weight torc

In [4]:
def replace(input, to_replace, replace_with):
    idx = input.find(to_replace)
    l = len(to_replace)
    return input[:idx] + replace_with + input[idx+l:]

for key, value in dict_1.items():        
    if 'decoder.layers.0' in key:
        print(key , value.shape)
print()
for n, p in model.named_parameters():
    if 'mamba_layers.0' in n:
        print(n, p.shape)

for n, p in model.named_parameters():
    if 'mamba_layers.' in n:
        name = 'decoder.layers.' + n[19:]
        if 'input_layernorm' in name:
            name = replace(name, 'input_layernorm', 'norm')
        else:
            name = replace(name, 'mamba', 'mixer')
        assert p.data.shape == dict_1[name].shape
        p.data = dict_1[name].clone().detach()

decoder.layers.0.mixer.dt_bias torch.Size([80])
decoder.layers.0.mixer.A_log torch.Size([80])
decoder.layers.0.mixer.D torch.Size([80])
decoder.layers.0.mixer.in_proj.0.weight torch.Size([10448, 2560])
decoder.layers.0.mixer.conv1d.weight torch.Size([5248, 1, 4])
decoder.layers.0.mixer.conv1d.bias torch.Size([5248])
decoder.layers.0.mixer.norm.weight torch.Size([5120])
decoder.layers.0.mixer.out_proj.weight torch.Size([2560, 5120])
decoder.layers.0.norm.weight torch.Size([2560])

model.mamba_layers.0.mamba.dt_bias torch.Size([80])
model.mamba_layers.0.mamba.A_log torch.Size([80])
model.mamba_layers.0.mamba.D torch.Size([80])
model.mamba_layers.0.mamba.in_proj.0.weight torch.Size([10448, 2560])
model.mamba_layers.0.mamba.conv1d.weight torch.Size([5248, 1, 4])
model.mamba_layers.0.mamba.conv1d.bias torch.Size([5248])
model.mamba_layers.0.mamba.norm.weight torch.Size([5120])
model.mamba_layers.0.mamba.out_proj.weight torch.Size([2560, 5120])
model.mamba_layers.0.input_layernorm.weight tor

In [16]:
for key, value in dict_1.items():
    if 'decoder.blocks.0' in key and 'extra_state' not in key:
        print(key, value.shape)
print()
for n, p in model.named_parameters():
    if 'blocks.0' in n:
        print(n, p.shape)

for n, p in model.named_parameters():
    if 'blocks.0.self_attn.linear_qkv.weight' in n:
        name = 'decoder.blocks.0.sa.mixer.linear_qkv.weight'
        assert p.data.shape == dict_1[name].shape
        p.data = dict_1[name].clone().detach()
    if 'blocks.0.self_attn.linear_proj.weight' in n:
        name = 'decoder.blocks.0.sa.mixer.linear_proj.weight'
        assert p.data.shape == dict_1[name].shape
        p.data = dict_1[name].clone().detach()
    if 'blocks.1.self_attn.linear_qkv.weight' in n:
        name = 'decoder.blocks.1.sa.mixer.linear_qkv.weight'
        assert p.data.shape == dict_1[name].shape
        p.data = dict_1[name].clone().detach()
    if 'blocks.1.self_attn.linear_proj.weight' in n:
        name = 'decoder.blocks.1.sa.mixer.linear_proj.weight'
        assert p.data.shape == dict_1[name].shape
        p.data = dict_1[name].clone().detach()

    if 'blocks.0.feed_forward.linear_fc1.weight' in n:
        name = 'decoder.blocks.0.mlp.mixer.linear_fc1.weight'
        assert p.data.shape == dict_1[name].shape
        p.data = dict_1[name].clone().detach()
    if 'blocks.0.feed_forward.linear_fc2.weight' in n:
        name = 'decoder.blocks.0.mlp.mixer.linear_fc2.weight'
        assert p.data.shape == dict_1[name].shape
        p.data = dict_1[name].clone().detach()
    if 'blocks.1.feed_forward.linear_fc1.weight' in n:
        name = 'decoder.blocks.1.mlp.mixer.linear_fc1.weight'
        assert p.data.shape == dict_1[name].shape
        p.data = dict_1[name].clone().detach()
    if 'blocks.1.feed_forward.linear_fc2.weight' in n:
        name = 'decoder.blocks.1.mlp.mixer.linear_fc2.weight'
        assert p.data.shape == dict_1[name].shape
        p.data = dict_1[name].clone().detach()
        
    if 'blocks.0.input_layernorm.weight' in n:
        name = 'decoder.blocks.0.sa.norm.weight'
        assert p.data.shape == dict_1[name].shape
        p.data = dict_1[name].clone().detach()
    if 'blocks.1.input_layernorm.weight' in n:
        name = 'decoder.blocks.1.sa.norm.weight'
        assert p.data.shape == dict_1[name].shape
        p.data = dict_1[name].clone().detach()
    if 'blocks.0.pre_ff_layernorm.weight' in n:
        name = 'decoder.blocks.0.mlp.norm.weight'
        assert p.data.shape == dict_1[name].shape
        p.data = dict_1[name].clone().detach()
    if 'blocks.1.pre_ff_layernorm.weight' in n:
        name = 'decoder.blocks.1.mlp.norm.weight'
        assert p.data.shape == dict_1[name].shape
        p.data = dict_1[name].clone().detach()
    if 'blocks.0.feed_forward.linear_fc1_lora_A' in n:
        num = replace(n, 'model.blocks.0.feed_forward.linear_fc1_lora_', '')[7]
        name = 'decoder.blocks.0.mlp.mixer.linear_fc1_lora' + replace('_A_list.0.weight', '0', num)
        assert p.data.shape == dict_1[name].shape
        p.data = dict_1[name].clone().detach()
    if 'blocks.0.feed_forward.linear_fc1_lora_B' in n:
        num = replace(n, 'model.blocks.0.feed_forward.linear_fc1_lora_', '')[7]
        name = 'decoder.blocks.0.mlp.mixer.linear_fc1_lora' + replace('_B_list.0.weight', '0', num)
        assert p.data.shape == dict_1[name].shape
        p.data = dict_1[name].clone().detach()
    if 'blocks.1.feed_forward.linear_fc1_lora_A' in n:
        num = replace(n, 'model.blocks.1.feed_forward.linear_fc1_lora_', '')[7]
        name = 'decoder.blocks.1.mlp.mixer.linear_fc1_lora' + replace('_A_list.0.weight', '0', num)
        assert p.data.shape == dict_1[name].shape
        p.data = dict_1[name].clone().detach()
    if 'blocks.1.feed_forward.linear_fc1_lora_B' in n:
        num = replace(n, 'model.blocks.1.feed_forward.linear_fc1_lora_', '')[7]
        name = 'decoder.blocks.1.mlp.mixer.linear_fc1_lora' + replace('_B_list.0.weight', '0', num)
        assert p.data.shape == dict_1[name].shape
        p.data = dict_1[name].clone().detach()

decoder.blocks.0.sa.mixer.linear_proj.weight torch.Size([2560, 5120])
decoder.blocks.0.sa.mixer.linear_qkv.weight torch.Size([15360, 5120])
decoder.blocks.0.sa.norm.weight torch.Size([5120])
decoder.blocks.0.mlp.mixer.linear_fc1.weight torch.Size([20480, 2560])
decoder.blocks.0.mlp.mixer.linear_fc2.weight torch.Size([2560, 10240])
decoder.blocks.0.mlp.mixer.linear_fc1_lora_A_list.0.weight torch.Size([128, 2560])
decoder.blocks.0.mlp.mixer.linear_fc1_lora_A_list.1.weight torch.Size([128, 2560])
decoder.blocks.0.mlp.mixer.linear_fc1_lora_A_list.2.weight torch.Size([128, 2560])
decoder.blocks.0.mlp.mixer.linear_fc1_lora_A_list.3.weight torch.Size([128, 2560])
decoder.blocks.0.mlp.mixer.linear_fc1_lora_A_list.4.weight torch.Size([128, 2560])
decoder.blocks.0.mlp.mixer.linear_fc1_lora_A_list.5.weight torch.Size([128, 2560])
decoder.blocks.0.mlp.mixer.linear_fc1_lora_A_list.6.weight torch.Size([128, 2560])
decoder.blocks.0.mlp.mixer.linear_fc1_lora_A_list.7.weight torch.Size([128, 2560])
dec

In [47]:
from transformers import AutoTokenizer
# Check forward pass
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
prompt_text = "Today is the first time that"
tokenized_prompt = tokenizer(prompt_text, return_tensors = 'pt').to("cuda")
print(tokenizer.decode(tokenized_prompt.input_ids.squeeze()))
output = model(tokenized_prompt.input_ids)
out_ids = torch.argmax(output.logits, dim=-1).squeeze()
print(tokenizer.decode(out_ids))

<s> Today is the first time that
<function MLP.__init__.<locals>.glu at 0x7f3307637b50>
tensor([[[-0.3457,  0.0161,  0.1758,  ..., -0.2480, -0.1328, -0.4062],
         [-0.3535,  0.0093,  0.0625,  ..., -0.0267, -0.0449, -0.1768],
         [-0.1167, -0.1162,  0.2227,  ..., -0.0854, -0.0908, -0.0547],
         ...,
         [ 0.0415, -0.3594, -0.2305,  ...,  0.2520, -0.0361, -0.0049],
         [-0.0762, -0.2373, -0.3398,  ...,  0.1445,  0.2275, -0.0957],
         [ 0.1162, -0.1562, -0.2852,  ...,  0.2031, -0.0752, -0.0024]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<AddBackward0>)
<function MLP.__init__.<locals>.glu at 0x7f330687ecb0>
tensor([[[-0.1514, -0.1504, -0.5312,  ...,  0.5117,  0.2129,  1.0781],
         [ 0.0017, -0.4336, -0.4023,  ...,  0.0776, -0.1865, -0.1187],
         [-0.0659, -0.2539, -0.3262,  ...,  0.3184,  0.0674, -0.0015],
         ...,
         [-0.4004, -0.2871, -0.3223,  ...,  0.3105, -0.1216,  0.2812],
         [-0.2656, -0.4922, -0.0581,  ...,  0.2

In [48]:
# SAVE WEIGHTS IN BIN FILE
BIN_SAVE_PATH = '/workspace/transformers_zamba2_dev/src/transformers/models/zamba2/state_dict_hf.bin'
torch.save(model.state_dict(), BIN_SAVE_PATH)

# Load model from .bin

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'
import torch
from transformers.models.zamba2 import Zamba2ForCausalLM, Zamba2Config
from transformers import AutoTokenizer


# Instantiate random Zamba
config = Zamba2Config()
model = Zamba2ForCausalLM(config).cuda()

weights = torch.load('/workspace/transformers_zamba2_dev/src/transformers/models/zamba2/state_dict_hf.bin')
model.load_state_dict(weights)
# model = model.to(torch.bfloat16)

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
prompt_text = "Today is the first time that"
tokenized_prompt = tokenizer(prompt_text, return_tensors = 'pt').to("cuda")
print(tokenizer.decode(tokenized_prompt.input_ids.squeeze()))
output = model(tokenized_prompt.input_ids)
print(output.logits.shape)
print(output.logits)
out_ids = torch.argmax(output.logits, dim=-1).transpose(0,1)
print(tokenizer.decode(out_ids.squeeze(0)))

Zamba requires an initialized `HybridMambaAttentionDynamicCache` to return a cache. None was provided, so no cache will be returned.


<s> Today is the first time that
torch.Size([7, 1, 32000])
tensor([[[-4.1799,  3.0531,  0.4019,  ..., -4.2610, -4.1060, -1.7605]],

        [[-7.2152,  6.8994, -1.9752,  ..., -6.3170, -5.7430, -4.1967]],

        [[-5.0736,  3.2236, -1.5486,  ..., -4.9293, -4.5989, -3.4123]],

        ...,

        [[-5.2612,  3.3268, -5.0263,  ..., -5.3279, -6.6721, -3.9966]],

        [[-6.1001,  4.9477, -0.8853,  ..., -5.5717, -6.0048, -4.3728]],

        [[-5.1238,  3.4259, -3.8259,  ..., -4.6562, -5.1366, -2.7277]]],
       device='cuda:0', grad_fn=<UnsafeViewBackward0>)


In [2]:
# Check forward pass

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
prompt_text = "Today is the first time that"
tokenized_prompt = tokenizer(prompt_text, return_tensors = 'pt').to("cuda")
print(tokenizer.decode(tokenized_prompt.input_ids.squeeze()))
output = model(tokenized_prompt.input_ids)
out_ids = torch.argmax(output.logits, dim=-1).squeeze()
print(tokenizer.decode(out_ids))

Zamba requires an initialized `HybridMambaAttentionDynamicCache` to return a cache. None was provided, so no cache will be returned.


<s> Today is the first time that
, the  day I I
